In [1]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('../data/data.csv')
np.set_printoptions(threshold=np.nan)

In [2]:
df_withlabel = df.copy()
df_withlabel['timely'] = 1

# type(df[(df.event=='begin payment') | (df.event=='finish payment') | (df.event=='abort payment')].groupby(['case']).groups)
casesHavingBeginPayment = df[df.event=='begin payment'].case.unique().tolist()
dftmp = df.groupby('case').first()
dftmp = dftmp.drop(list(set(dftmp.index.values) - set(casesHavingBeginPayment)))
# alternative for above 2 lines, but muuuuuuuuuuuuuuuuuch slower
# dftmp = df[df.case.isin(casesHavingBeginPayment)]
sy = pd.DatetimeIndex(dftmp.groupby('case').first().startTime).year.tolist()
cy = pd.DatetimeIndex(df[df.event=='begin payment'].groupby('case').first().startTime).year.tolist()

diffIdx = []
for pos, val in enumerate(sy):
    if val != cy[pos]:
        diffIdx.append(pos)
late_begin_payment_indexes = df[df.case.isin(dftmp.iloc[diffIdx].index.values)].index.values
df_withlabel.loc[late_begin_payment_indexes, 'timely'] = 0
late_begin_payment_indexes = None # delete useless variable
sy = None
cy = None

In [3]:
no_finish_payment_list = list(set(df.case) - set(df[df.event=='finish payment'].case.unique().tolist()))
print('There are', len(no_finish_payment_list), 'cases having no \'finish payment\' event')
no_finish_payment_indexes = df[df.case.isin(no_finish_payment_list)].index.values
df_withlabel.loc[no_finish_payment_indexes, 'timely'] = 0

There are 88 cases having no 'finish payment' event


In [4]:
dftmp = df.iloc[list(set(df.index.values.tolist()) - set(no_finish_payment_indexes.tolist()))]
by = pd.DatetimeIndex(dftmp[dftmp.event=='begin payment'].groupby('case').first().startTime).year.tolist()
fy = pd.DatetimeIndex(df[df.event=='finish payment'].groupby('case').first().startTime).year.tolist()

In [5]:
diffIdx = []
for pos, val in enumerate(by):
#     print(val, fy[pos])
    if val != fy[pos]:
        diffIdx.append(pos)
print('There are', len(diffIdx), 'cases having: 这个 case 第一个 event 所记录的那年出现 begin payment 但是当年没有 finish payment')

There are 43557 cases having: 这个 case 第一个 event 所记录的那年出现 begin payment 但是当年没有 finish payment


In [6]:
df_withlabel.loc[(~np.isnan(df_withlabel.payment_actual1)) | (~np.isnan(df_withlabel.payment_actual2)) | (~np.isnan(df_withlabel.payment_actual3)), 'timely'] = 0

In [7]:
case = df_withlabel.drop(columns=['subprocess', 'org:resource', 'event_identity:id', 'eventid', 'docid_uuid', 'docid', 'doctype', 'success', 'activity', 'note', 'startTime', 'completeTime', 'event', 'basic payment', 'program-id', 'greening', 'identity:id', 'application', 'applicant']+[i+str(j) for i in ['amount_applied','payment_actual', 'penalty_amount'] for j in [1,2,3]]).drop_duplicates().fillna(0).set_index(['case'])
dpt_to_numerical = dict((dpt, i) for i, dpt in enumerate(case.department.unique()))
case.department = case.department.map(dpt_to_numerical)

In [8]:
case.shape

(43809, 47)

In [9]:
label = case.values[:, -1].astype('?')
data = case.values[:, :-1]
print('One data sample:')
display(case.values[0])

One data sample:


array([False, False, False, False, False, False, False, False, 960.35,
       False, 2015, False, 960.35, False, False, False, False, False,
       False, False, False, False, 0.0, False, False, False, False, 0,
       False, True, False, False, False, 2.6994, 1.0, False, True, 0.0,
       False, 3, False, False, False, False, False, False, 1],
      dtype=object)

In [2]:
import os
import numpy as np
if not os.path.exists('./data/store.npz'):
    np.savez('./data/store.npz',label=label,data=data)
else:
    store = np.load('./data/store.npz')
    label = store['label']
    data = store['data']

In [11]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
params = {'criterion':['gini'], 'min_samples_split': [1100]}
dtree = RandomForestClassifier(n_estimators=2000, random_state=117, n_jobs=-1)
clf = GridSearchCV(dtree, params, cv=5, verbose=1)
clf.fit(data, label)
print (clf.best_score_, clf.best_params_)
print(clf.cv_results_)
# clf = DecisionTreeClassifier(random_state=0)
# cross_val_score(clf, data, label, cv=10)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  1.1min finished


0.9325709329133283 {'criterion': 'gini', 'min_samples_split': 1100}
{'mean_fit_time': array([10.95435886]), 'std_fit_time': array([0.25162404]), 'mean_score_time': array([0.64839072]), 'std_score_time': array([0.04177923]), 'param_criterion': masked_array(data=['gini'],
             mask=[False],
       fill_value='?',
            dtype=object), 'param_min_samples_split': masked_array(data=[1100],
             mask=[False],
       fill_value='?',
            dtype=object), 'params': [{'criterion': 'gini', 'min_samples_split': 1100}], 'split0_test_score': array([0.92809861]), 'split1_test_score': array([0.93038119]), 'split2_test_score': array([0.93209313]), 'split3_test_score': array([0.93266378]), 'split4_test_score': array([0.93961876]), 'mean_test_score': array([0.93257093]), 'std_test_score': array([0.00386527]), 'rank_test_score': array([1], dtype=int32), 'split0_train_score': array([0.93391731]), 'split1_train_score': array([0.93334665]), 'split2_train_score': array([0.93283305])